In [7]:
# Importing dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

test_df = pd.read_csv('../Resources/test_clean.csv')
train_df = pd.read_csv('../Resources/train_clean.csv')

display(test_df)
display(train_df)

,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,892,0,3,Kelly,Mr James,male,34.5,0,0,330911,7.8292,Q,"D,E,F,G"
1,893,1,3,Wilkes,Mrs James (Ellen Needs),female,47.0,1,0,363272,7.0000,S,"D,E,F,G"
2,894,0,2,Myles,Mr Thomas Francis,male,62.0,0,0,240276,9.6875,Q,"D,E,F,G"
3,895,0,3,Wirz,Mr Albert,male,27.0,0,0,315154,8.6625,S,"D,E,F,G"
4,896,1,3,Hirvonen,Mrs Alexander (Helga E Lindqvist),female,22.0,1,1,3101298,12.2875,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,Spector,Mr Woolf,male,23.0,0,0,A.5. 3236,8.0500,S,"D,E,F,G"
414,1306,1,1,Oliva y Ocana,Dona Fermina,female,39.0,0,0,PC 17758,108.9000,C,"A,B,C,D,E"
415,1307,0,3,Saether,Mr Simon Sivertsen,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S,"D,E,F,G"
416,1308,0,3,Ware,Mr Frederick,male,34.0,0,0,359309,8.0500,S,"D,E,F,G"


,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,1,0,3,Braund,Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,S,"D,E,F,G"
1,2,1,1,Cumings,Mrs John Bradley (Florence Briggs Thayer),female,38.0,1,0,PC 17599,71.2833,C,"A,B,C,D,E"
2,3,1,3,Heikkinen,Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,S,"D,E,F,G"
3,4,1,1,Futrelle,Mrs Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1000,S,"A,B,C,D,E"
4,5,0,3,Allen,Mr William Henry,male,35.0,0,0,373450,8.0500,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,887,0,2,Montvila,Rev Juozas,male,27.0,0,0,211536,13.0000,S,"D,E,F,G"
885,888,1,1,Graham,Miss Margaret Edith,female,19.0,0,0,112053,30.0000,S,"A,B,C,D,E"
886,889,0,3,Johnston,Miss Catherine Helen Carrie,female,7.0,1,2,W./C. 6607,23.4500,S,"D,E,F,G"
887,890,1,1,Behr,Mr Karl Howell,male,26.0,0,0,111369,30.0000,C,"A,B,C,D,E"


In [8]:
# Dropping non-beneficial ID columns
test_df_dropped = test_df.drop(['PassengerId', 'last_name', 'first_name'], axis=1)
train_df_dropped = train_df.drop(['PassengerId', 'last_name', 'first_name'], axis=1)

display(test_df_dropped)
display(train_df_dropped)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,0,3,male,34.5,0,0,330911,7.8292,Q,"D,E,F,G"
1,1,3,female,47.0,1,0,363272,7.0000,S,"D,E,F,G"
2,0,2,male,62.0,0,0,240276,9.6875,Q,"D,E,F,G"
3,0,3,male,27.0,0,0,315154,8.6625,S,"D,E,F,G"
4,1,3,female,22.0,1,1,3101298,12.2875,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...
413,0,3,male,23.0,0,0,A.5. 3236,8.0500,S,"D,E,F,G"
414,1,1,female,39.0,0,0,PC 17758,108.9000,C,"A,B,C,D,E"
415,0,3,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S,"D,E,F,G"
416,0,3,male,34.0,0,0,359309,8.0500,S,"D,E,F,G"


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,0,3,male,22.0,1,0,A/5 21171,7.2500,S,"D,E,F,G"
1,1,1,female,38.0,1,0,PC 17599,71.2833,C,"A,B,C,D,E"
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,"D,E,F,G"
3,1,1,female,35.0,1,0,113803,53.1000,S,"A,B,C,D,E"
4,0,3,male,35.0,0,0,373450,8.0500,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...
884,0,2,male,27.0,0,0,211536,13.0000,S,"D,E,F,G"
885,1,1,female,19.0,0,0,112053,30.0000,S,"A,B,C,D,E"
886,0,3,female,7.0,1,2,W./C. 6607,23.4500,S,"D,E,F,G"
887,1,1,male,26.0,0,0,111369,30.0000,C,"A,B,C,D,E"


In [9]:
# Looking at the number of unique values in each column for test_df
print(test_df_dropped.nunique())

Survived      2
Pclass        3
Sex           2
Age          80
SibSp         7
Parch         8
Ticket      363
Fare        169
Embarked      3
Deck          2
dtype: int64


In [10]:
# Looking at the number of unique values in each column for train_df
print(train_df_dropped.nunique())

Survived      2
Pclass        3
Sex           2
Age          89
SibSp         7
Parch         7
Ticket      679
Fare        248
Embarked      3
Deck          2
dtype: int64


In [11]:
train_df_dropped['Ticket'].value_counts()

Ticket
1601        7
CA. 2343    7
347082      7
CA 2144     6
347088      6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: count, Length: 679, dtype: int64

In [12]:
train_dummies_df = pd.get_dummies(train_df_dropped)
train_dummies_df

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Ticket_110152,Ticket_110413,...,Ticket_W./C. 6608,Ticket_W./C. 6609,Ticket_W.E.P. 5734,Ticket_W/C 14208,Ticket_WE/P 5735,Embarked_C,Embarked_Q,Embarked_S,"Deck_A,B,C,D,E","Deck_D,E,F,G"
0,0,3,22.0,1,0,7.2500,False,True,False,False,...,False,False,False,False,False,False,False,True,False,True
1,1,1,38.0,1,0,71.2833,True,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2,1,3,26.0,0,0,7.9250,True,False,False,False,...,False,False,False,False,False,False,False,True,False,True
3,1,1,35.0,1,0,53.1000,True,False,False,False,...,False,False,False,False,False,False,False,True,True,False
4,0,3,35.0,0,0,8.0500,False,True,False,False,...,False,False,False,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,0,2,27.0,0,0,13.0000,False,True,False,False,...,False,False,False,False,False,False,False,True,False,True
885,1,1,19.0,0,0,30.0000,True,False,False,False,...,False,False,False,False,False,False,False,True,True,False
886,0,3,7.0,1,2,23.4500,True,False,False,False,...,False,False,False,False,False,False,False,True,False,True
887,1,1,26.0,0,0,30.0000,False,True,False,False,...,False,False,False,False,False,True,False,False,True,False


In [13]:
# Spltting preprocessed data into features and target arrays
X = train_dummies_df.copy()
X.drop('Survived', axis=1, inplace=True)
y = train_dummies_df['Survived']

# Splitting data into training and testing
X_train, X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)